# Exploring Reddit Submission Votes and Comments

In [1]:
import praw
import pandas as pd
import datetime
import json

In [2]:
# Load credfile and display when last updated
credfile = 'credfile.json'
credfile_prefix = ''

# Read credentials to a dictionary
with open(credfile) as fh:
    creds = json.loads(fh.read())

print(f"[{datetime.datetime.now()}]" + f"{credfile} {'.' * 10} is being used as credfile")

reddit = praw.Reddit(client_id=creds['client_id'],
                     client_secret=creds['client_secret'],
                     user_agent=creds['user_agent']
                    )

print(reddit.read_only)  # Output: True

[2019-04-09 22:38:40.287344]credfile.json .......... is being used as credfile
True


### Look at the current 10 'hot' submissions from r/AskReddit

In [3]:
subreddit = reddit.subreddit('AskReddit')
hot_10 = list(subreddit.hot(limit=10))
hot_10

[Submission(id='bbbuxc'),
 Submission(id='bb90vv'),
 Submission(id='bb7s99'),
 Submission(id='bb804d'),
 Submission(id='bb6vue'),
 Submission(id='bbalma'),
 Submission(id='bb7qsi'),
 Submission(id='bb5tv8'),
 Submission(id='bb7joj'),
 Submission(id='bb9bcr')]

In [4]:
controv_10 = list(subreddit.controversial(limit=10))
top_10 = list(subreddit.top(limit=10))
new_10 = list(subreddit.new(limit=10))

#### Comments for an example submission

In [5]:
submission = reddit.submission(id='bavxqq')
submission.url

'https://www.reddit.com/r/AskReddit/comments/bavxqq/all_crimes_are_now_legal_for_24_hours_and_you/'

In [6]:
# To iterate through all instances of MoreComments to get all top level comments
while True:
    try:
        submission.comments.replace_more()
        break
    except PossibleExceptions:
        print('Handling replace_more exception')
        sleep(1)

In [7]:
print(len(list(submission.comments
              )))

1349


#### View comment replies

In [8]:
comment = submission.comments[5]
comment.refresh()
comment.replies.replace_more()

[]

In [9]:
replies = comment.replies

In [10]:
list(replies)

[Comment(id='ekf2xdv'),
 Comment(id='ekf6v4f'),
 Comment(id='ekfkxw9'),
 Comment(id='ekfyx5o'),
 Comment(id='ekg2611'),
 Comment(id='ekghem6'),
 Comment(id='ekgp98i')]

### Comments & Upvotes

In [13]:
def iterate_over_submissions(submission_list, selection):
    submission_df = pd.DataFrame()
    counter=0
    for submission in submission_list:
        while True:
            try:
                submission.comments.replace_more()
                break
            except PossibleExceptions:
                print('Handling replace_more exception')
                sleep(1)
                
        if selection=="print":
            print("Name of post:       ", submission.name)
            print("Author:             ", submission.author)
            print("Created on:         ", datetime.datetime.fromtimestamp(int(submission.created_utc)).strftime("%m/%d/%y %H:%M:%S"))
            print("Number of upvotes:  ", submission.ups)
            print("Linked url:         ", submission.url)
            print("Upvote ratio:       ", submission.upvote_ratio)
            print("Number of comments: ", len(list(submission.comments)))
            print("=========================")
            
        elif selection=="dataframe":
            submission_df.loc[counter, 'name'] = submission.name
            submission_df.loc[counter, 'created'] = datetime.datetime.fromtimestamp(int(submission.created_utc)).strftime("%m/%d/%y %H:%M:%S")
            submission_df.loc[counter, 'upvotes'] = submission.ups
            submission_df.loc[counter, 'ratio'] = submission.upvote_ratio
            submission_df.loc[counter, 'comments'] = len(list(submission.comments))
            
    return submission_df

In [15]:
iterate_over_submissions(controv_10, selection="print")

Name of post:        t3_fn67j
Author:              None
Created on:          02/17/11 08:13:49
Number of upvotes:   112
Linked url:          https://www.reddit.com/r/AskReddit/comments/fn67j/i_hear_my_house_mate_masturbating_should_i_walk/
Upvote ratio:        0.55
Number of comments:  1348
Name of post:        t3_gajxc
Author:              SmashPig
Created on:          03/24/11 11:56:26
Number of upvotes:   0
Linked url:          https://www.reddit.com/r/AskReddit/comments/gajxc/today_is_my_birthday_i_turned_25_all_i_want_for/
Upvote ratio:        0.5
Number of comments:  103
Name of post:        t3_x6kv0
Author:              xomaleo
Created on:          07/26/12 05:42:22
Number of upvotes:   168
Linked url:          https://www.reddit.com/r/AskReddit/comments/x6kv0/why_do_americans_refer_to_their_country_as_free/
Upvote ratio:        0.57
Number of comments:  1777
Name of post:        t3_u843q
Author:              tttouch_me
Created on:          05/27/12 22:32:14
Number of upvotes:  

""


In [16]:
submission_df = iterate_over_submissions(controv_10, selection="dataframe")

### TODO: Vizualization of comments vs upvotes